Property Graph from Arrows App

### Property Graph Design

![Property Graph Design](../images/arrows_design.svg)

Creating in Cypher:

```
CREATE (:Person {readUser: "", gender: "", age: "", ageGroup: ""})-[:INVOLVED_IN]->(n0:Crash {crashID: "", crashType: "", numberFatalities: "", busInvolvement: "", heavyRigidTruckInvolvement: "", articulatedTruckInvolvement: "", speedLimit: ""})-[:OCCURED_AT]->(:Location {state: "", nationalRemoteAreas: "", `sa4Name`: "", lgaName: "", nationalRoadType: ""}),
(n0)-[:HAPPENED_AT]->(:DateTime {month: "", year: "", dayOfWeek: "", time: "", timeOfDay: "", weekdayOrWeekend: "", christmasPeriod: "", easterPeriod: ""})
```

### ETL Process to Create CSVs for Neo4j Import
We'll need to prepare 4 CSV files:

1. `crash_nodes.csv` - All crash-related properties
2. `person_nodes.csv` - All person-related properties  
3. `location_nodes.csv` - All location-related properties
4. `dateTime_nodes.csv` - All time-related properties

And 3 relationship CSV files:

1. `person_crash_rel.csv` - Connecting persons to crashes
2. `crash_location_rel.csv` - Connecting crashes to locations  
3. `crash_timeframe_rel.csv` - Connecting crashes to timeframes

In [21]:
import pandas as pd
import os

# Define input and output paths
input_file = '../data/original/Project2_Dataset_Corrected.csv'  # Change this to your input CSV file name
output_dir = '../data/filtered'  # Directory to store output CSV files

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [22]:
# Read the original CSV file
print(f"Reading data from {input_file}...")
df = pd.read_csv(input_file)
print(f"Found {len(df)} records")

Reading data from ../data/original/Project2_Dataset_Corrected.csv...
Found 10490 records


In [23]:
# Create unique IDs for nodes
df['personId'] = df['ID'].astype(str)  # Using the original ID as personId
df['locationId'] = df['Crash ID'].astype(str) + '_' + df['State'] + '_' + df['National Remoteness Areas'] + '_' + df['SA4 Name 2021'] + '_' + df['National LGA Name 2024'] + '_' + df['National Road Type']
df['dateTimeId'] = df['Crash ID'].astype(str) + '_' + df['Month'].astype(str) + '_' + df['Year'].astype(str) + '_' + df['Time']

# Prepare dataframes for each node type
# Person nodes
person_df = df[['personId', 'Road User', 'Gender', 'Age', 'Age Group', 'Crash ID']].copy()
person_df.columns = ['personId', 'roadUser', 'gender', 'age', 'ageGroup', 'crashId']

# Crash nodes
crash_df = df[['Crash ID', 'Crash Type', 'Number Fatalities', 'Bus Involvement', 
               'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement', 
               'Speed Limit']].copy()
crash_df.columns = ['crashId', 'crashType', 'numberFatalities', 'busInvolvement', 
                    'heavyRigidTruckInvolvement', 'articulatedTruckInvolvement', 'speedLimit']

# Remove duplicates to ensure unique crash nodes
crash_df = crash_df.drop_duplicates(subset=['crashId'])

# Location nodes
location_df = df[['locationId', 'State', 'National Remoteness Areas', 'SA4 Name 2021', 
                  'National LGA Name 2024', 'National Road Type', 'Crash ID']].copy()
location_df.columns = ['locationId', 'state', 'nationalRemoteAreas', 'sa4Name', 
                       'lgaName', 'nationalRoadType', 'crashId']

# Remove duplicates to ensure unique location nodes
location_df = location_df.drop_duplicates(subset=['locationId'])

# DateTime nodes
dateTime_df = df[['dateTimeId', 'Month', 'Year', 'Dayweek', 'Time', 'Time of day', 
                  'Day of week', 'Christmas Period', 'Easter Period', 'Crash ID']].copy()
dateTime_df.columns = ['dateTimeId', 'month', 'year', 'dayOfWeek', 'time', 'timeOfDay', 
                      'weekdayOrWeekend', 'christmasPeriod', 'easterPeriod', 'crashId']

# Remove duplicates to ensure unique datetime nodes
dateTime_df = dateTime_df.drop_duplicates(subset=['dateTimeId'])


In [24]:

# Prepare relationship dataframes
# For INVOLVED_IN relationship - Person to Crash
person_crash_rel = person_df[['personId', 'crashId']].copy()

# For OCCURED_AT relationship - Crash to Location
crash_location_rel = location_df[['crashId', 'locationId']].copy()

# For HAPPENED_AT relationship - Crash to DateTime
crash_dateTime_rel = dateTime_df[['crashId', 'dateTimeId']].copy()


In [25]:
# Export node CSVs
person_df.to_csv(f"{output_dir}/person_nodes.csv", index=False)
crash_df.to_csv(f"{output_dir}/crash_nodes.csv", index=False)
location_df.to_csv(f"{output_dir}/location_nodes.csv", index=False)
dateTime_df.to_csv(f"{output_dir}/dateTime_nodes.csv", index=False)

# Export relationship CSVs
person_crash_rel.to_csv(f"{output_dir}/person_crash_rel.csv", index=False)
crash_location_rel.to_csv(f"{output_dir}/crash_location_rel.csv", index=False)
crash_dateTime_rel.to_csv(f"{output_dir}/crash_dateTime_rel.csv", index=False)


### Cypher commands

In [26]:
cypher_commands = f"""
// Create constraints for unique IDs
CREATE CONSTRAINT crash_id IF NOT EXISTS FOR (c:Crash) REQUIRE c.crashId IS UNIQUE;
CREATE CONSTRAINT person_id IF NOT EXISTS FOR (p:Person) REQUIRE p.personId IS UNIQUE;
CREATE CONSTRAINT location_id IF NOT EXISTS FOR (l:Location) REQUIRE l.locationId IS UNIQUE;
CREATE CONSTRAINT dateTime_id IF NOT EXISTS FOR (d:DateTime) REQUIRE d.dateTimeId IS UNIQUE;

// Import Crash nodes
LOAD CSV WITH HEADERS FROM 'file:///crash_nodes.csv' AS row
CREATE (c:Crash {{
  crashId: row.crashId,
  crashType: row.crashType,
  numberFatalities: toInteger(row.numberFatalities),
  busInvolvement: row.busInvolvement,
  heavyRigidTruckInvolvement: row.heavyRigidTruckInvolvement,
  articulatedTruckInvolvement: row.articulatedTruckInvolvement,
  speedLimit: toInteger(row.speedLimit)
}});

// Import Person nodes
LOAD CSV WITH HEADERS FROM 'file:///person_nodes.csv' AS row
CREATE (p:Person {{
  personId: row.personId,
  roadUser: row.roadUser,
  gender: row.gender,
  age: toInteger(row.age),
  ageGroup: row.ageGroup
}});

// Import Location nodes
LOAD CSV WITH HEADERS FROM 'file:///location_nodes.csv' AS row
CREATE (l:Location {{
  locationId: row.locationId,
  state: row.state,
  nationalRemoteAreas: row.nationalRemoteAreas,
  sa4Name: row.sa4Name,
  lgaName: row.lgaName,
  nationalRoadType: row.nationalRoadType
}});

// Import DateTime nodes
LOAD CSV WITH HEADERS FROM 'file:///dateTime_nodes.csv' AS row
CREATE (d:DateTime {{
  dateTimeId: row.dateTimeId,
  month: toInteger(row.month),
  year: toInteger(row.year),
  dayOfWeek: row.dayOfWeek,
  time: row.time,
  timeOfDay: row.timeOfDay,
  weekdayOrWeekend: row.weekdayOrWeekend,
  christmasPeriod: row.christmasPeriod,
  easterPeriod: row.easterPeriod
}});

// Create Person-Crash relationships (INVOLVED_IN)
LOAD CSV WITH HEADERS FROM 'file:///person_crash_rel.csv' AS row
MATCH (p:Person {{personId: row.personId}})
MATCH (c:Crash {{crashId: row.crashId}})
CREATE (p)-[:INVOLVED_IN]->(c);

// Create Crash-Location relationships (OCCURED_AT)
LOAD CSV WITH HEADERS FROM 'file:///crash_location_rel.csv' AS row
MATCH (c:Crash {{crashId: row.crashId}})
MATCH (l:Location {{locationId: row.locationId}})
CREATE (c)-[:OCCURED_AT]->(l);

// Create Crash-DateTime relationships (HAPPENED_AT)
LOAD CSV WITH HEADERS FROM 'file:///crash_dateTime_rel.csv' AS row
MATCH (c:Crash {{crashId: row.crashId}})
MATCH (d:DateTime {{dateTimeId: row.dateTimeId}})
CREATE (c)-[:HAPPENED_AT]->(d);

// Create indexes for better performance
CREATE INDEX crash_speed_idx IF NOT EXISTS FOR (c:Crash) ON (c.speedLimit);
CREATE INDEX crash_articulated_idx IF NOT EXISTS FOR (c:Crash) ON (c.articulatedTruckInvolvement);
CREATE INDEX crash_fatalities_idx IF NOT EXISTS FOR (c:Crash) ON (c.numberFatalities);
CREATE INDEX location_state_idx IF NOT EXISTS FOR (l:Location) ON (l.state);
CREATE INDEX dateTime_year_idx IF NOT EXISTS FOR (d:DateTime) ON (d.year);
CREATE INDEX person_age_group_idx IF NOT EXISTS FOR (p:Person) ON (p.ageGroup);
CREATE INDEX person_road_user_idx IF NOT EXISTS FOR (p:Person) ON (p.roadUser);
"""

In [28]:
# Save Cypher commands to a file
with open(f"{output_dir}/neo4j_import_commands.txt", "w") as f:
    f.write(cypher_commands)

print(f"Neo4j import commands saved to {output_dir}/neo4j_import_commands.txt")
print("ETL process complete!")

Neo4j import commands saved to ../data/filtered/neo4j_import_commands.txt
ETL process complete!
